In [1]:
import numpy as np
import pandas as pd

from libreco.data import DatasetPure
from libreco.algorithms import BPR, WideDeep
from libreco.evaluation import evaluate

from sklearn.model_selection import train_test_split

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
!pip install numpy pandas scikit-learn tqdm scipy openai

In [3]:
! pip install -U LibRecommender

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Preprocess Data

In [9]:
books_df = pd.read_csv('/content/drive/MyDrive/Books_Extended.csv', delimiter=';', low_memory=False)
ratings_df = pd.read_csv('/content/drive/MyDrive/Ratings.csv', delimiter=';')
users_df = pd.read_csv('/content/drive/MyDrive/Users.csv', delimiter=';')

<ipython-input-9-497a94a7983d>:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users_df = pd.read_csv('/content/drive/MyDrive/Users.csv', delimiter=';')


In [10]:
ratings_df.columns = ['user_id', 'item_id', 'label']

In [3]:
df_clened_with_emb = pd.read_csv('/content/df_with_emb_cleaned.csv')

# Train / Eval Split

In [4]:
train_df, eval_df = train_test_split(df_clened_with_emb, test_size=0.2, random_state=0)

In [5]:
pca_columns = [f'pca_dim_{i+1}' for i in range(10)]

In [6]:
columns = ['user', 'item', 'label']
user_cols = ['Age']
item_cols = pca_columns + ['Year']

columns = columns + user_cols + item_cols

In [7]:
eval_user_mask = eval_df['user'].isin(train_df['user'].unique())
eval_item_mask = eval_df['item'].isin(train_df['item'].unique())

eval_df = eval_df[eval_user_mask & eval_item_mask]

# 'user', 'item' must be the first two columns of the dataframe
train_df = train_df[columns]
eval_df = eval_df[columns]

In [8]:
from libreco.data import DataInfo, DatasetFeat, split_by_ratio
train_data, data_info = DatasetFeat.build_trainset(train_df, user_col=user_cols, item_col=item_cols, dense_col=item_cols + user_cols)
test_data = DatasetFeat.build_testset(eval_df)

# Bayesian Personalized Ranking and Wide Deep


In [9]:
embedding_size = 32
n_epochs_bpr = 400
n_epochs_wide = 400
learning_rate = 0.01
batch_size = 256
random_seed = 42

In [10]:
# init BPR model
bpr = BPR(
    task="ranking",
    data_info=data_info,
    embed_size=embedding_size,
    n_epochs=n_epochs_bpr,
    lr=learning_rate,
    reg=None,
    batch_size=batch_size,
    num_neg=1,
    use_tf=True,
    seed=random_seed
)

In [11]:
# train BPR model
#from tensorflow.compat.v1 import reset_default_graph
#reset_default_graph()

bpr.fit(
    train_data,
    neg_sampling=True,
    verbose=2,
    shuffle=True,
    eval_data=test_data,
    metrics=["ndcg", "precision"]
)

Training start time: 2024-08-11 19:04:24


train: 100%|██████████| 33/33 [00:00<00:00, 176.75it/s]


Epoch 1 elapsed: 0.190s
	 train_loss: 0.6972


eval_listwise: 100%|██████████| 227/227 [00:00<00:00, 1954.48it/s]


	 eval ndcg@10: 0.0422
	 eval precision@10: 0.0084


...

	 eval ndcg@10: 0.0136
	 eval precision@10: 0.0031


train: 100%|██████████| 33/33 [00:00<00:00, 467.54it/s]


Epoch 400 elapsed: 0.082s
	 train_loss: 0.0408


eval_listwise: 100%|██████████| 227/227 [00:00<00:00, 1510.64it/s]

	 eval ndcg@10: 0.0155
	 eval precision@10: 0.0040


In [13]:
# evaluate the BPR model
bpr_eval_result = evaluate(bpr, test_data, neg_sampling=True, metrics=["ndcg", "precision", "recall"])
print(f"Evaluation Results (BPR):\n{bpr_eval_result}")

eval_listwise: 100%|██████████| 227/227 [00:00<00:00, 1019.58it/s]

Evaluation Results (BPR):
{'ndcg': 0.015458543071106596, 'precision': 0.003964757709251102, 'recall': 0.021125350420504604}
